Suppose we want to create the an integrand in QMCPy for evaluating the following integral: 
    $$ \int_{[0,1]^d} \|x\|_2^{\|x\|_2^{1/2}} dx,$$
where $[0,1]^d$ is the unit hypercube in $\mathbb{R}^d$. The integrand is defined everywhere except at $x=0$ and hence the definite integal is also defined.
    
    
The key in defining a Python function of an integrand in the QMCPy framework is that not only  it should be able to take one point $x \in \mathbb{R}^d$ and return a real value, but also that it would be able to take a set of $n$ sampling points as rows in a Numpy array of size $n \times d$ and return an array with $n$ values evaluated at each sampling point. The following examples illustrate this point.

In [1]:
from qmcpy import *
from numpy.linalg import norm as norm
from numpy import sqrt, array

Our first attempt maybe to create the integrand as a Python function as follows:

In [2]:
def f(x): return norm(x) ** sqrt(norm(x))

It looks reasonable except that maybe the Numpy function norm is executed twice. It's okay for now. Let us quickly test if the function behaves as expected at a point value:

In [3]:
x = 0.01
f(x)

0.6309573444801932

What about an array that represents $n=3$ sampling points in a two-dimensional domain, i.e., $d=2$? 

In [4]:
x = array([[1, 0], 
           [0, 0.01],
           [0.04, 0.04]])

In [5]:
f(x)

1.001650000560437

Now, the function should have returned $n=3$ real values that correponding to each of the sampling points. Let's debug our Python function.

In [6]:
norm(x)

1.0016486409914407

Numpy's norm(x) is obviously a matrix norm, but we want it to be vector 2-norm that acts on each row of x. To that end, let's add an axis argument to the function:

In [7]:
norm(x, axis = 1)

array([ 1.000,  0.010,  0.057])

Now it's working! Let's make sure that the sqrt function is acting on each element of the vector norm results:

In [8]:
sqrt(norm(x, axis = 1))

array([ 1.000,  0.100,  0.238])

It is. Putting everything together, we have:

In [9]:
norm(x, axis = 1) ** sqrt(norm(x, axis = 1))

array([ 1.000,  0.631,  0.505])

We have got our proper function definition now.

In [10]:
def f(x):
    x_norms = norm(x, axis = 1)
    return x_norms ** sqrt(x_norms)

We can now create an integrand instance with our QuickConstruct class in QMCPy and then invoke QMCPy's integrate function:

In [11]:
integrand = QuickConstruct(custom_fun=f)
sol, data = integrate(integrand, Uniform(1))
print(data)

Solution: 0.6596         
QuickConstruct (Integrand Object)
	fun_str         def f(x):
	                   x_norms = norm(x, axis = 1)
	                   return x_norms ** sqrt(x_norms)
IIDStdUniform (Discrete Distribution Object)
	mimics          StdUniform
	rng_seed        None
Uniform (True Measure Object)
	dimension       1
	a               0
	b               1
CLT (Stopping Criterion Object)
	abs_tol         0.010
	rel_tol         0
	n_max           10000000000
	alpha           0.010
	inflate         1.200
MeanVarData (AccumData Object)
	n               3232
	n_total         4256.0
	confid_int      [ 0.649  0.670]
	time_total      0.004



For our integral, we know the true value. Let's check if QMCPy's solution is accurate enough:

In [12]:
true_sol = 0.658582  # In WolframAlpha: Integral[x**Sqrt[x], {x,0,1}]
abs_tol = data.stopping_criterion.abs_tol
qmcpy_error = abs(true_sol - sol)
print(qmcpy_error < abs_tol)

True


It's good. Shall we test the function with $d=2$ by simply changing the input parameter value of dimension for QuickConstruct?

In [13]:
integrand2 = QuickConstruct(custom_fun=f)
sol2, data2 = integrate(integrand2, Uniform(2))
print(data2)

Solution: 0.8278         
QuickConstruct (Integrand Object)
	fun_str         def f(x):
	                   x_norms = norm(x, axis = 1)
	                   return x_norms ** sqrt(x_norms)
IIDStdUniform (Discrete Distribution Object)
	mimics          StdUniform
	rng_seed        None
Uniform (True Measure Object)
	dimension       2
	a               0
	b               1
CLT (Stopping Criterion Object)
	abs_tol         0.010
	rel_tol         0
	n_max           10000000000
	alpha           0.010
	inflate         1.200
MeanVarData (AccumData Object)
	n               5425
	n_total         6449.0
	confid_int      [ 0.818  0.838]
	time_total      0.002



Once again, we could test for accuracy of QMCPy with respect to the true value:

In [14]:
true_sol2 = 0.827606  # In WolframAlpha: Integral[Sqrt[x**2+y**2])**Sqrt[Sqrt[x**2+y**2]], {x,0,1}, {y,0,1}]
abs_tol2 = data2.stopping_criterion.abs_tol
qmcpy_error2 = abs(true_sol2 - sol2)
print(qmcpy_error2 < abs_tol2)

True


## Keister Example

In [15]:
integrand = Keister()
discrete_distrib = Sobol(rng_seed=7)
true_measure = Gaussian(dimension=3, variance=1 / 2)
stopping_criterion = CLTRep(discrete_distrib, true_measure, abs_tol=.05)
_, data = integrate(integrand, true_measure, discrete_distrib, stopping_criterion)
print(data)

Solution: 2.1477         
Keister (Integrand Object)
Sobol (Discrete Distribution Object)
	mimics          StdUniform
	rng_seed        7
Gaussian (True Measure Object)
	dimension       3
	mu              0
	sigma           0.707
CLTRep (Stopping Criterion Object)
	abs_tol         0.050
	rel_tol         0
	n_max           1048576
	alpha           0.010
	inflate         1.200
MeanVarDataRep (AccumData Object)
	n               32
	n_total         32
	confid_int      [ 2.126  2.169]
	time_total      0.008
	r               16



## Asian Option Pricing Example

In [16]:
# Singl-Level
time_vec = [arange(1 / 64, 65 / 64, 1 / 64)]
dim = [len(tv) for tv in time_vec]

discrete_distrib = Lattice(rng_seed=7)
true_measure = BrownianMotion(dim, time_vector=time_vec)
integrand = AsianCall(true_measure)
stopping_criterion = CLTRep(discrete_distrib, true_measure, abs_tol=.05)
_, data = integrate(integrand, true_measure, discrete_distrib, stopping_criterion)
print(data)

Solution: 6.2021         
AsianCall (Integrand Object)
	volatility      0.500
	start_price     30
	strike_price    25
	interest_rate   0
	mean_type       arithmetic
	exercise_time   1.0
Lattice (Discrete Distribution Object)
	mimics          StdUniform
	rng_seed        7
BrownianMotion (True Measure Object)
	dimension       64
	time_vector     [[ 0.016  0.031  0.047 ...  0.969  0.984  1.000]]
CLTRep (Stopping Criterion Object)
	abs_tol         0.050
	rel_tol         0
	n_max           1048576
	alpha           0.010
	inflate         1.200
MeanVarDataRep (AccumData Object)
	n               2048
	n_total         2048
	confid_int      [ 6.200  6.204]
	time_total      0.473
	r               16



In [17]:
# Multi-Level
time_vec = [arange(1 / 4, 5 / 4, 1 / 4),
                 arange(1 / 16, 17 / 16, 1 / 16),
                 arange(1 / 64, 65 / 64, 1 / 64)]
dim = [len(tv) for tv in time_vec]

discrete_distrib = IIDStdGaussian(rng_seed=7)
true_measure = BrownianMotion(dim, time_vector=time_vec)
integrand = AsianCall(true_measure,
                volatility = .5,
                start_price = 30,
                strike_price = 25,
                interest_rate = .01,
                mean_type = 'geometric')
stopping_criterion = CLT(discrete_distrib, true_measure, abs_tol=.05, n_max = 1e10)
_, data = integrate(integrand, true_measure, discrete_distrib, stopping_criterion)
print(data)

Solution: 5.8335         
AsianCall (Integrand Object)
	volatility      0.500
	start_price     30
	strike_price    25
	interest_rate   0.010
	mean_type       geometric
	exercise_time   1.0
IIDStdGaussian (Discrete Distribution Object)
	mimics          StdGaussian
	rng_seed        7
BrownianMotion (True Measure Object)
	dimension       [ 4 16 64]
	time_vector     [array([ 0.250,  0.500,  0.750,  1.000])
	                array([ 0.062,  0.125,  0.188, ...,  0.875,  0.938,  1.000])
	                array([ 0.016,  0.031,  0.047, ...,  0.969,  0.984,  1.000])]
CLT (Stopping Criterion Object)
	abs_tol         0.050
	rel_tol         0
	n_max           10000000000
	alpha           0.010
	inflate         1.200
MeanVarData (AccumData Object)
	n               [ 248458.000  33175.000  5226.000]
	n_total         289931.0
	confid_int      [ 5.785  5.882]
	time_total      0.152

